# <strong>YZV 202E - OPTIMIZATION FOR DATA SCIENCE PROJECT</strong>
# Importance of Optimization Techniques in Post-Earthquake Relief

## Team: Iron-Flag
## Team Members: Mustafa Bayrak, Zehra Demir

Install necessary packages

In [82]:
pip install basemap

Note: you may need to restart the kernel to use updated packages.Requirement already satisfied: pyshp<2.4,>=1.2 in c:\users\hp\anaconda3\lib\site-packages (from basemap) (2.3.1)



In [ ]:
pip install geopy

### Importing the necessary libraries

In [ ]:
import pandas as pd
import numpy as np
from mpl_toolkits.basemap import Basemap
import matplotlib.pyplot as plt
import random
from geopy.geocoders import Nominatim
random.seed(1773)

### Reading the Excel file that contains populations and information of whether they are affected city or distribution city

In [ ]:
df = pd.read_excel("../Datasets/excel/cities.xlsx")

### First 15 cities in the dataframe (descending order)

In [ ]:
df.head(15)

### Load Arrays of distribution centers and affected cities

In [ ]:
dist_centers = np.load('../Datasets/numpy-arrays/dist_cities.npy')
affected_cities = np.load('../Datasets/numpy-arrays/affected_cities.npy')

### Names of distribution centers

In [ ]:
dist_centers

### Names of the affected cities

In [ ]:
affected_cities

### Creating the distances matrix

In [ ]:
distances = pd.read_excel("../Datasets/excel/ilmesafe.xlsx")

In [ ]:
distances

Create a new dataframe containing distribution cities in rows

In [ ]:
new_df = pd.DataFrame(columns=distances.columns)

# Iterate over each row in the original dataframe
for index, row in distances.iterrows():
    if row['Name'] in dist_centers:
        # Append the row to the new dataframe
        new_df = pd.concat([new_df, row.to_frame().T])

# Reset the index of the new dataframe
new_df.reset_index(drop=True, inplace=True)

### Distances between distribution centers to each affected city

In [ ]:
distance_df = new_df[affected_cities]

In [ ]:
distance_df

In [ ]:
distance_matrix = np.array(distance_df)

In [ ]:
# Geocoding setup
geolocator = Nominatim(user_agent="my-app")

# Map setup
map = Basemap(llcrnrlon=26, llcrnrlat=35, urcrnrlon=45, urcrnrlat=42, resolution='l')
map.drawcoastlines()

# Plot red cities
for city in dist_centers:
    location = geolocator.geocode(city + ", Turkey")
    lon, lat = location.longitude, location.latitude
    x, y = map(lon, lat)
    map.plot(x, y, 'bo')

# Plot blue cities
for city in affected_cities:
    location = geolocator.geocode(city + ", Turkey")
    lon, lat = location.longitude, location.latitude
    x, y = map(lon, lat)
    map.plot(x, y, 'ro')

# Show the map
plt.show()

## Fixed Values

In [ ]:
MAX_HELICOPTER_CAPACITY = 20
HELICOPTER_SPEED = 200
MAX_TRUCK_CAPACITY = 50
TRUCK_SPEED = 100
HELICOPTER_NUMBER = 10
TRUCK_NUMBER = 500
NUM_DISTRIBUTION_CENTERS = 10
NUM_AFFECTED_CITIES = 11

### Supplies and demands are proportional with population

In [ ]:
supplies = np.load('../Datasets/numpy-arrays/supplies.npy')

demands = np.load('../Datasets/numpy-arrays/demands.npy')

In [ ]:
max_helicopter = np.load('../Datasets/numpy-arrays/max_helicopter.npy')

In [ ]:
max_truck = np.load('../Datasets/numpy-arrays/max_truck.npy')

# Genetic Algorithm

### Creating the Population

In [ ]:
def create_population(size, max_truck, max_helicopter, supplies, demands):
    population = []
    for _ in range(size):
        # Copy maximum capacities
        remaining_truck = max_truck.copy()
        remaining_helicopter = max_helicopter.copy()
        
        # Shuffle the order of indices for i and j
        indices_i = list(range(10))
        indices_j = list(range(11))
        
        # Shuffle the order of indices again for i and j
        random.shuffle(indices_i)
        random.shuffle(indices_j)

        # Create a 2D list for truck transfers with random values
        truck_transfers = np.zeros((10, 11), dtype=int)
        for i in indices_i:
            for j in indices_j:
                # Maximum transfer allowed is the lesser of remaining supply, demand, and remaining truck capacity
                max_transfer = min(supplies[i], demands[j], remaining_truck[i])  
                truck_transfer = np.random.randint(0, max_transfer + 1)
                truck_transfers[i][j] = truck_transfer
                # Decrease remaining truck capacity
                remaining_truck[i] -= truck_transfer
                
        random.shuffle(indices_i)
        random.shuffle(indices_j)
        
        # Create a 2D list for helicopter transfers based on truck transfers
        helicopter_transfers = np.zeros((10, 11), dtype=int)
        for i in indices_i:
            for j in indices_j:
                # Maximum transfer allowed is the lesser of remaining supply, remaining demand, and remaining helicopter capacity
                remaining_supply = supplies[i] - truck_transfers[i][j]
                remaining_demand = demands[j] - truck_transfers[i][j]
                max_transfer = min(remaining_supply, remaining_demand, remaining_helicopter[i])  
                helicopter_transfer = np.random.randint(0, max_transfer + 1)  # Generate random helicopter transfer
                helicopter_transfers[i][j] = helicopter_transfer
                # Decrease remaining helicopter capacity
                remaining_helicopter[i] -= helicopter_transfer
        individual = [
            truck_transfers,
            helicopter_transfers
        ]
        population.append(individual)
    return population

### Fitness Function

In [ ]:
def fitness_function(individual):
    truck_transfers = np.array(individual[0])
    helicopter_transfers = np.array(individual[1])
    total_cost = np.sum(distance_matrix * (helicopter_transfers / HELICOPTER_SPEED))\
    + np.sum(distance_matrix * (truck_transfers / TRUCK_SPEED))
    penalty = 0
    for i in range(len(supplies)):
        if (sum(truck_transfers[i]) + sum(helicopter_transfers[i])) > supplies[i] * 1.2:
            penalty +=1
        elif sum(truck_transfers[i]) > max_truck[i] *1.2\
        or sum(helicopter_transfers[i]) > max_helicopter[i] *1.2:
            penalty +=1
    total_cost += 1e6 *penalty
    return -total_cost

### Selection

In [ ]:
def selection(population, num_parents, fitnesses):
    # calculate total fitness of all individuals
    total_fitness = sum(fitnesses)
    # calculate relative fitness of each individual
    rel_fitness = [f/total_fitness for f in fitnesses]
    # generate probability intervals for each individual
    probs = [sum(rel_fitness[:i+1]) for i in range(len(rel_fitness))]
    # draw new population
    new_population = []
    for n in range(num_parents):
        r = random.random()
        for (i, individual) in enumerate(population):
            if r <= probs[i]:
                new_population.append(individual)
                break
    return new_population

### Crossover

In [ ]:
def uniform_crossover(parent1, parent2):
    # Define a crossover rate
    crossover_rate = 0.5

    # Perform uniform crossover
    child1_truck = []
    child2_truck = []
    child1_heli = []
    child2_heli = []

    # Crossover for truck transfers
    for i in range(len(parent1[0])):
        if np.random.random() < crossover_rate:
            child1_truck.append(parent1[0][i])
            child2_truck.append(parent2[0][i])
        else:
            child1_truck.append(parent2[0][i])
            child2_truck.append(parent1[0][i])

    # Crossover for helicopter transfers
    for i in range(len(parent1[1])):
        if np.random.random() < crossover_rate:
            child1_heli.append(parent1[1][i])
            child2_heli.append(parent2[1][i])
        else:
            child1_heli.append(parent2[1][i])
            child2_heli.append(parent1[1][i])
    
    return [np.array(child1_truck), np.array(child1_heli)], [np.array(child2_truck), np.array(child2_heli)]

### Mutation

In [ ]:
def mutation(individual, mutation_rate, max_truck, max_helicopter, supplies, demands):
    truck_transfers, helicopter_transfers = individual

    for i in range(10):
        for j in range(11):
            if np.random.rand() < mutation_rate:
                    truck_transfers[i][j] += np.random.randint(-20,20)
                    helicopter_transfers[i][j] += np.random.randint(-20,20)
                    if truck_transfers[i][j] <0:
                        truck_transfers[i][j] = 0
                    if helicopter_transfers[i][j] <0:
                        helicopter_transfers[i][j] = 0

    return [truck_transfers, helicopter_transfers]

### Implementation of the Algorithm

In [ ]:
def genetic_algorithm(population_size, num_generations, mutation_rate):
    # Create initial population
    population = create_population(population_size, max_truck, max_helicopter, supplies, demands)
    
    # Evaluate the population
    fitness_values = [fitness_function(individual) for individual in population]

    # Iterate through each generation
    for generation in range(num_generations):

        # Create a new population
        new_population = []

        while len(new_population) < population_size:
            # Select two parents
            parent1, parent2 = selection(population, 2, fitness_values)
            
            # Create two children by crossover
            child1, child2 = uniform_crossover(parent1, parent2)

            # Add the children to the new population
            new_population += [child1, child2]

        # Apply mutation
        new_population = [mutation(individual, mutation_rate, max_truck, max_helicopter, supplies, demands) for individual in new_population]

        # Replace the old population with the new population
        population = new_population

        # Evaluate the new population
        fitness_values = [fitness_function(individual) for individual in population]
        
    # Return the individual with the best fitness value
    best_fitness_index = np.argmax(fitness_values)
    best_individual = population[best_fitness_index]

    return best_individual



population_size = 10
num_generations = 200
mutation_rate = 0.01

best_individual = genetic_algorithm(population_size, num_generations, mutation_rate)

## Results of Genetic Algorithm

In [ ]:
print("Best Individual(Truck):\n", best_individual[0],"\n")
print("Best Individual(Helicopter):\n", best_individual[1])
print("Best Fitness:", -fitness_function(best_individual))

# Simulated Annealing Algorithm

### Creating the Population

In [ ]:
def create_population(size, max_truck, max_helicopter, supplies, demands):
    population = []
    for _ in range(size):
        # Copy maximum capacities
        remaining_truck = max_truck.copy()
        remaining_helicopter = max_helicopter.copy()
        
        # Shuffle the order of indices for i and j
        indices_i = list(range(10))
        indices_j = list(range(11))
        
        # Shuffle the order of indices again for i and j
        random.shuffle(indices_i)
        random.shuffle(indices_j)

        # Create a 2D list for truck transfers with random values
        truck_transfers = np.zeros((10, 11), dtype=int)
        for i in indices_i:
            for j in indices_j:
                # Maximum transfer allowed is the lesser of remaining supply, demand, and remaining truck capacity
                max_transfer = min(supplies[i], demands[j], remaining_truck[i])  
                truck_transfer = np.random.randint(0, max_transfer + 1)
                truck_transfers[i][j] = truck_transfer
                # Decrease remaining truck capacity
                remaining_truck[i] -= truck_transfer
                
        random.shuffle(indices_i)
        random.shuffle(indices_j)
        
        # Create a 2D list for helicopter transfers based on truck transfers
        helicopter_transfers = np.zeros((10, 11), dtype=int)
        for i in indices_i:
            for j in indices_j:
                # Maximum transfer allowed is the lesser of remaining supply, remaining demand, and remaining helicopter capacity
                remaining_supply = supplies[i] - truck_transfers[i][j]
                remaining_demand = demands[j] - truck_transfers[i][j]
                max_transfer = min(remaining_supply, remaining_demand, remaining_helicopter[i])  
                helicopter_transfer = np.random.randint(0, max_transfer + 1)  # Generate random helicopter transfer
                helicopter_transfers[i][j] = helicopter_transfer
                # Decrease remaining helicopter capacity
                remaining_helicopter[i] -= helicopter_transfer
        individual = [
            truck_transfers,
            helicopter_transfers
        ]
        population.append(individual)
    return population

### Fitness Function

In [ ]:
def fitness_function(individual):
    truck_transfers = np.array(individual[0])
    helicopter_transfers = np.array(individual[1])
    total_cost = np.sum(distance_matrix * (helicopter_transfers / HELICOPTER_SPEED))\
    + np.sum(distance_matrix * (truck_transfers / TRUCK_SPEED))
    penalty = 0
    for i in range(len(supplies)):
        if (sum(truck_transfers[i]) + sum(helicopter_transfers[i])) > supplies[i] * 1.2:
            penalty +=1
        elif sum(truck_transfers[i]) > max_truck[i] *1.2\
        or sum(helicopter_transfers[i]) > max_helicopter[i] *1.2:
            penalty +=1
    total_cost += 1e6 *penalty
    return -total_cost

In [ ]:
def random_neighbor(individual, max_truck, max_helicopter, supplies, demands):
    truck_transfers, helicopter_transfers = individual
    
    # Shuffle the order of indices for i and j
    indices_i = list(range(10))
    indices_j = list(range(11))
        
    # Shuffle the order of indices again for i and j
    random.shuffle(indices_i)
    random.shuffle(indices_j)
    
    for i in indices_i:
        for j in indices_j:
            if random.random() < 0.5:
                if sum(truck_transfers[i]) >= max_truck[i]:
                    truck_transfers[i][j] -= np.random.randint(0, 30)
                    if truck_transfers[i][j] < 0:
                        truck_transfers[i][j] = 0
                else:
                    truck_transfers[i][j] += np.random.randint(-2, 10)
                    if truck_transfers[i][j] < 0:
                        truck_transfers[i][j] = 0
            if random.random() < 0.5:
                if sum(helicopter_transfers[i]) >= max_helicopter[i]:
                    helicopter_transfers[i][j] -= np.random.randint(0, 30)
                    if helicopter_transfers[i][j] < 0:
                        helicopter_transfers[i][j] = 0
                else:
                    helicopter_transfers[i][j] += np.random.randint(-2, 10)
                    if helicopter_transfers[i][j] < 0:
                        helicopter_transfers[i][j] = 0 

    # Check if the new solution exceeds capacity limits
    for i in range(len(supplies)):
        if (sum(truck_transfers[i]) + sum(helicopter_transfers[i])) > supplies[i] * 1.2 \
                or sum(truck_transfers[i]) > max_truck[i] \
                or sum(helicopter_transfers[i]) > max_helicopter[i]:
            return individual  # Return the original solution if the new solution exceeds capacity limits

    return [truck_transfers, helicopter_transfers]

### Probability

In [ ]:
def acceptance_probability(energy, new_energy, temperature):
    if new_energy < energy:
        return 1.0
    return np.exp((energy - new_energy) / temperature)

### Implementation of the Algorithm

In [ ]:
def simulated_annealing(initial_solution, max_truck, max_helicopter, supplies, demands, num_iterations, max_temperature):
    current_solution = initial_solution
    best_solution = initial_solution
    current_fitness = fitness_function(current_solution)
    best_fitness = current_fitness

    for iteration in range(num_iterations):
        temperature = max_temperature * (1 - iteration / num_iterations)  # Cooling schedule

        new_solution = random_neighbor(current_solution, max_truck, max_helicopter, supplies, demands)
        new_fitness = fitness_function(new_solution)

        if acceptance_probability(current_fitness, new_fitness, temperature) > random.random():
            current_solution = new_solution
            current_fitness = new_fitness

        if new_fitness > best_fitness:
            best_solution = new_solution
            best_fitness = new_fitness

    return best_solution

In [ ]:
population_size = 1000
initial_solution = create_population(population_size, max_truck, max_helicopter, supplies, demands)[0]
num_iterations = 1000
max_temperature = 10

best_individual = simulated_annealing(initial_solution, max_truck, max_helicopter, supplies, demands,
                                     num_iterations, max_temperature)

## Results

In [ ]:
print("Best Individual (Truck):\n", best_individual[0], "\n")
print("Best Individual (Helicopter):\n", best_individual[1])
print("Best Fitness:", -fitness_function(best_individual))